# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [86]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [87]:
pth1 = '/datasets/users_behavior.csv'
pth2 = '/Users/Alex/Desktop/practicum_ds/projects/ml/users_behavior.csv'

if os.path.exists(pth1):
    df = pd.read_csv(pth1)
elif os.path.exists(pth2):
    df = pd.read_csv(pth2)
else:
    print('Something is wrong')

In [88]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [90]:
df['is_ultra'].unique()

array([0, 1])

**Вывод:**

Итак, в таблице 11 столбецов.

Описание данных:

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

сalls — количество звонков,

minutes — суммарная длительность звонков в минутах,

messages — количество sms-сообщений,

mb_used — израсходованный интернет-трафик в Мб,

is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

Предварительно можно утверждать, что, данных достаточно для решения задачи.

Количество значений в столбцах не различается. Значит, в данных нет пропусков.

В первом приближении артефакты (аномалии)* не обнаружены (значения, которые не отражают действительность и появились по какой-то ошибке).

Соблюдение требований к стилю в названии колонок:

Строчные буквы - да.

Отсутствие пробелов - да.

Использован «змеиный_регистр» (snake_case) - да.

## Разбейте данные на выборки

In [91]:
df_train, df_valid_plus_test = train_test_split(
    df,
    test_size=0.4,
    random_state=12345,
    stratify=df['is_ultra'])

In [92]:
df_valid, df_test = train_test_split(
    df_valid_plus_test,
    test_size=0.5,
    random_state=12345,
    stratify=df_valid_plus_test['is_ultra'])

In [93]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

## Исследуйте модели

**Решающее дерево**

In [94]:
tree_best_model = None
tree_best_result = 0
max_depth = 0
for depth in range(1, 11):
    tree_model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    tree_model.fit(features_train, target_train)
    tree_result = tree_model.score(features_valid, target_valid)
    if tree_result > tree_best_result:
        tree_best_model = tree_model
        tree_best_result = tree_result
        tree_max_depth = depth

print("Accuracy наилучшей модели на валидационной выборке:", tree_best_result, ', при max_depth =', tree_max_depth)
print(tree_best_model)

Accuracy наилучшей модели на валидационной выборке: 0.80248833592535 , при max_depth = 8
DecisionTreeClassifier(max_depth=8, random_state=12345)


**Cлучайный лес**

- с перебором глубины деревьев

In [95]:
forest_tree_best_model = None
forest_tree_best_result = 0
for estim in range(1, 11):
    for depth_1 in range(1, 11):
        forest_tree_model = RandomForestClassifier(random_state=12345, n_estimators=estim, max_depth=depth_1)
        forest_tree_model.fit(features_train, target_train)
        forest_tree_result = forest_tree_model.score(features_valid, target_valid)
        if forest_tree_result > forest_tree_best_result:
            forest_tree_best_model = forest_tree_model
            forest_tree_best_result = forest_tree_result
            forest_tree_n_estimators = estim
            forest_tree_max_depth = depth_1

print("Accuracy наилучшей модели на валидационной выборке:",
      forest_tree_best_result, ', n_estimators =', forest_tree_n_estimators,
      ', при max_depth =', forest_tree_max_depth)
print(forest_tree_best_model)

Accuracy наилучшей модели на валидационной выборке: 0.807153965785381 , n_estimators = 6 , при max_depth = 6
RandomForestClassifier(max_depth=6, n_estimators=6, random_state=12345)


**Логистическая регрессия**

In [96]:
log_model = LogisticRegression(random_state=12345)
log_model.fit(features_train, target_train)
log_result = log_model.score(features_valid, target_valid)


print("Accuracy наилучшей модели на валидационной выборке:", log_result)
print(log_model)

Accuracy наилучшей модели на валидационной выборке: 0.7387247278382582
LogisticRegression(random_state=12345)


**Качество (Accuracy) лучшей модели в Решающем дереве и Случайном лесе (c перебором глубины деревьев) практически равны, при этом в Логистической регрессии данный показатель значительно ниже.**

**При равенстве показателя качества, показатель скорости у Решающего дерева выше.**

Проверим, что изменится при проверке лучших моделей Решающего дерева и Случайного леса на тестовой выборке.

## Проверьте модель на тестовой выборке

**Решающее дерево**

In [97]:
print("Accuracy наилучшей модели на тестовой выборке:",
      tree_best_model.score(features_test, target_test),
      ', при max_depth =', tree_max_depth)
print(tree_best_model)

Accuracy наилучшей модели на тестовой выборке: 0.7838258164852255 , при max_depth = 8
DecisionTreeClassifier(max_depth=8, random_state=12345)


**Cлучайный лес**

- с перебором глубины деревьев

In [98]:
print("Accuracy наилучшей модели на тестовой выборке:",
      forest_tree_best_model.score(features_test, target_test),
      ', n_estimators =', forest_tree_n_estimators,
      ', при max_depth =', forest_tree_max_depth)
print(forest_tree_best_model)

Accuracy наилучшей модели на тестовой выборке: 0.8102643856920684 , n_estimators = 6 , при max_depth = 6
RandomForestClassifier(max_depth=6, n_estimators=6, random_state=12345)


**Логистическая регрессия**

In [99]:
print("Accuracy наилучшей модели на тестовой выборке:",
      log_model.score(features_test, target_test))
print(log_model)

Accuracy наилучшей модели на тестовой выборке: 0.7465007776049767
LogisticRegression(random_state=12345)


In [101]:
print('Процент отставания модели Решающее дерево '
      'в показателе качества (Accuracy) на тестовой выборке '
      'в сравнении с моделью Случайный лес (с перебором глубины деревьев)',
      ((tree_best_model.score(features_test, target_test)/
        forest_tree_best_model.score(features_test, target_test)-1)*100).round(2),'%')

Процент отставания модели Решающее дерево в показателе качества (Accuracy) на тестовой выборке в сравнении с моделью Случайный лес (с перебором глубины деревьев) -3.26 %


**По итогам тестирования на тестовой выборке модель Случайный лес (с перебором глубины деревьев) показала самый лучший показатель качества (accuracy) и опередила модель Решающее дерево на 3,26%.**

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
